In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import base64
import math
import plotly.graph_objects as go

from functions.functions_data import get_global_daily, get_global_daily_v2, get_rolling, get_rolling_values
from functions.functions_graphics import plot_engagements_users, plot_metrics, plot_engagements_users_comparative
from functions.functions_data import get_engagement_list_v2, get_rolling_v2


In [5]:

#function to get group of age
def get_group_age(number):
    if 18 <= number <= 29:
        return "18-29"
    elif 30 <= number <= 39:
        return "30-39"
    elif 40 <= number <= 49:
        return "40-49"
    elif 50 <= number <= 59:
        return "50-59"
    elif 60 <= number <= 69:
        return "60-69"
    elif 70 <= number <= 79:
        return "70-79"
    else:
        return "80+"

In [6]:
#read the data
path_to_csv = "data/data_20230619.csv"
df = pd.read_csv(path_to_csv, parse_dates=['EventDateTime'], dtype={'CurrentPlatform': str,
                                                                    'CurrentType': str,
                                                                    'Platform': str,
                                                                    'Version': str})
#change data type
df['EventDateTime'] = pd.to_datetime(df['EventDateTime'])
#import csv with total balance 
path_to_csv_total_balance = "data/20230609-users.csv"
df_total_balance = pd.read_csv(path_to_csv_total_balance)
df_total_balance = df_total_balance.rename(columns = {'userId':'UserId'})
#merge csv
df = df.merge(df_total_balance, left_on='UserId', right_on='UserId', how='left')    
#User_client
#if totalBalance is greater 3 then is a Client, else is an User
df['User_Client'] = df['totalBalance'].apply(lambda x: 'Client' if x > 3 else 'User')
#get group of age
df["Group_age"] = df["Age"].apply(lambda x: get_group_age(x))
#Abreviaturas de los estados de USA
df_states = pd.read_csv("data/abreviaturas_USA.csv")


In [7]:
engagement_list = get_engagement_list_v2(df = df, start_date= str("2023-03-15"), end_data= str("2023-06-15")  )


In [11]:
global_metrics_v2 = get_global_daily_v2(engagement_list[engagement_list["UTMCampaign"].isin(["applaunch","APP_EN_IRA_RET_ACCOUNT-CREATION-FAILURE"])], "UTMCampaign")
rolled_v2 = get_rolling_v2(global_metrics_v2,int(7), df, "UTMCampaign")

In [22]:
def plot_engagements_users_comparative_v2(dataset, label, columna, filtros):
  """
  dataset: DataFrame
  label: str
  columna: name of the column for groupby
  filtros: active filtros of column

  Plot a figure with the Engagements and Unique Users.
  """
  # Initialize figure
  fig = go.Figure()

  # Add Traces
  data = dataset.copy()

  for f in filtros:
    data_temp= data[data[columna] == f]
    random_color= "#"+''.join([random.choice('0123456789ABCDEF') for i in range(6)])
    fig.add_trace(go.Scatter(x=data_temp["EventDateTime"], 
      y=data_temp["Engagements"],
      name="Engagements " + f, 
      mode="lines", 
      line=dict(color=random_color)
      ))

    fig.add_trace(go.Scatter(x=data_temp["EventDateTime"], 
      y=data_temp["Unique_users"], 
      name="Unique_users "+ f, 
      mode="lines", 
      line=dict(color=random_color, dash="dash")
      ))

  # Set title
  fig.update_layout(title_text=f"{label} comparative by {columna}")

  return fig

In [23]:
plot_engagements_users_comparative_v2(rolled_v2.reset_index(), str(7) +' days', "UTMCampaign", ["applaunch","APP_EN_IRA_RET_ACCOUNT-CREATION-FAILURE"])

#C44CB4
#AFE009


In [49]:
utmCampaign_list = df.UTMCampaign.value_counts().reset_index()
utmCampaign_list = list(utmCampaign_list["UTMCampaign"])

In [47]:
df.UTMCampaign.value_counts().reset_index()

,index,UTMCampaign
0,applaunch,291435
1,APP_ES_RET_AUTODEPOSIT_STREAK-PUSH,9026
2,APP_ES_RET_USER_WEEKLY_SUMMARY,7777
3,APP_ES_RET_DIVIDENDS_RECEIVED,6181
4,APP_ES_RET_AUTODEPOSIT_RECEIVED,5047
...,...,...
645,20220308_Upate_Info_ENG,1
646,APP_EN_TAX_RET_ITIN-3-EMAIL,1
647,APP_EN_RET_INVESTMENT-SELECTION-1-EMAIL,1
648,email_EN_20230309_IRAaccount,1
